In [1]:
#importing required libraries
%matplotlib inline
import os
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import geopy
import geocoder
import requests
from pandas.io.json import json_normalize
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported')

Libraries imported


## Scraping pincodes and post offices  in Banglore

In [2]:
venues=[]
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        28.725828,  
        77.162787, 
        500, 
        5000)
    
results = requests.get(url).json()["response"]['groups'][0]['items']
index=0
for venue in results:
    index+=1
    venues.append((
        index,
        venue['venue']['name'], 
        venue['venue']['location']['lat'], 
        venue['venue']['location']['lng'],  
        venue['venue']['categories'][0]['name']))
print(venues)

[(1, 'Jahangirpuri Metro Station | जहांगीरपुरी', 28.726025752225787, 77.16262578964233, 'Light Rail Station'), (2, 'Bluebird Pure Private Limited', 28.72523544228677, 77.1623682975769, 'Kitchen Supply Store')]


In [3]:
#scraping makaan.in for housing prices
df=pd.DataFrame(columns=['Name','Price range per sqft','Avg price per sqft','Price rise','Trend','View Properties'])
res=pd.read_html('https://www.makaan.com/price-trends/property-rates-for-buy-in-delhi',header=1)
res[0].columns=['Name','Price range per sqft','Avg price per sqft','Price rise','Trend','View Properties']
df=df.append(res[0],ignore_index=True)
for n in np.arange(2,59):
    res=pd.read_html('https://www.makaan.com/price-trends/property-rates-for-buy-in-delhi?page={}'.format(n),header=1)
    res[0].columns=['Name','Price range per sqft','Avg price per sqft','Price rise','Trend','View Properties']
    df=df.append(res[0],ignore_index=True)

In [4]:
df

,Name,Price range per sqft,Avg price per sqft,Price rise,Trend,View Properties
0,Uttam Nagar,"843 - 13,333 / sqft","7,486 / sqft",5.4%,See trend,View 468 properties
1,Uttam Nagar west,"996 - 11,548 / sqft","6,762.32 / sqft",-25.7%,See trend,View 98 properties
2,Malviya Nagar,"2,310 - 14,222 / sqft","11,703.7 / sqft",41.1%,See trend,View 12 properties
3,laxmi nagar,"2,353 - 13,889 / sqft","11,858.86 / sqft",-16.3%,See trend,View 588 properties
4,Greater kailash 1,"2,900 - 43,333 / sqft","17,797.51 / sqft",-15%,See trend,View 8 properties
5,Saket,"3,435 - 18,621 / sqft","13,995.16 / sqft",-7%,See trend,View 53 properties
6,Defence Colony,"758 - 35,484 / sqft","20,939.86 / sqft",21.8%,See trend,View 25 properties
7,Safdarjung Enclave,"6,500 - 29,423 / sqft","21,743.02 / sqft",-20.9%,See trend,View 12 properties
8,Vasant Kunj,"2,165 - 1,85,714 / sqft","68,576.87 / sqft",7%,See trend,View 248 properties
9,Greater Kailash II,"3,343 - 18,765 / sqft","16,788.9 / sqft",53.4%,See trend,View 9 properties


In [5]:
#removing unecessary columns price range per sqft, trends,price rise,view properties
df.drop(['Price range per sqft','Price rise','Trend','View Properties'],axis=1,inplace=True)
df

,Name,Avg price per sqft
0,Uttam Nagar,"7,486 / sqft"
1,Uttam Nagar west,"6,762.32 / sqft"
2,Malviya Nagar,"11,703.7 / sqft"
3,laxmi nagar,"11,858.86 / sqft"
4,Greater kailash 1,"17,797.51 / sqft"
5,Saket,"13,995.16 / sqft"
6,Defence Colony,"20,939.86 / sqft"
7,Safdarjung Enclave,"21,743.02 / sqft"
8,Vasant Kunj,"68,576.87 / sqft"
9,Greater Kailash II,"16,788.9 / sqft"


In [6]:
#dropping rows with - as their avg price per sqft
df=df[df['Avg price per sqft']!='-'].reset_index(drop=True)
df

,Name,Avg price per sqft
0,Uttam Nagar,"7,486 / sqft"
1,Uttam Nagar west,"6,762.32 / sqft"
2,Malviya Nagar,"11,703.7 / sqft"
3,laxmi nagar,"11,858.86 / sqft"
4,Greater kailash 1,"17,797.51 / sqft"
5,Saket,"13,995.16 / sqft"
6,Defence Colony,"20,939.86 / sqft"
7,Safdarjung Enclave,"21,743.02 / sqft"
8,Vasant Kunj,"68,576.87 / sqft"
9,Greater Kailash II,"16,788.9 / sqft"


In [7]:
#removing '/sqft' from end of the prices and ',' from values and converting it into float
avgPrice=[]
for price in df['Avg price per sqft'].values:
    price=price.replace(' / sqft','')
    price=price.replace(',','')
    avgPrice.append(price)
df['Avg price per sqft']=avgPrice

In [8]:
#converting Avg price per sqft column into float and renaming columns
df.rename(columns={'Avg price per sqft':'AvgPrice','Name':'AreaName'},inplace=True)
#converting names into lowercase
df['AreaName']=df['AreaName'].str.lower()
df.astype({'AvgPrice': 'float'}).dtypes

AreaName     object
AvgPrice    float64
dtype: object

In [9]:
#removing rows with 'near' in it because it will not lead to an specific location instead vicinity of location
df=df[~df.AreaName.str.contains('near')].reset_index(drop=True)
df

,AreaName,AvgPrice
0,uttam nagar,7486
1,uttam nagar west,6762.32
2,malviya nagar,11703.7
3,laxmi nagar,11858.86
4,greater kailash 1,17797.51
5,saket,13995.16
6,defence colony,20939.86
7,safdarjung enclave,21743.02
8,vasant kunj,68576.87
9,greater kailash ii,16788.9


In [10]:
#exporting dataframe to csv
df.to_csv('pricing.csv')